In [1]:
import pandas as pd
import itertools
final_df = pd.read_csv("train.csv")
for i in final_df.columns:
    final_df[i] = final_df[i].astype('str')

In [2]:
final_df = final_df.drop(['Unnamed: 0'],axis=1)
final_df

,Title,Client,SSN,Address,Ext_text
0,ACCOUNT TRANSFER FORM,Alexis W Figueroa,555455545,"1376 Brentwood Drive, Austin, Texas, 78701",Reset Form TD Ameritrade ACCOUNT TRANSFER FORM...
1,ACCOUNT TRANSFER FORM,Claudia M McMullen,44190820,"2966 Bedford Street, New York, Connecticut, 10011",['Reset Form TD Ameritrade ACCOUNT TRANSFER FO...
2,ACCOUNT TRANSFER FORM,David T Jarvis,372525158,"4572 Haven Lane, Lansing, Michigan,48933",['Reset Form TD Ameritrade ACCOUNT TRANSFER FO...
3,ACCOUNT TRANSFER FORM,Kelly D Ellis,153472012,"4819 Jadewood Farms,Newton, New Jersey, 07860",['Reset Form TD Ameritrade ACCOUNT TRANSFER FO...
4,ACCOUNT TRANSFER FORM,Marco S Ng,621729927,"2957 Patterson Road, Bronx,New York, 10461",['Reset Form TD Ameritrade ACCOUNT TRANSFER FO...


In [3]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import spacy
from spacy.util import decaying
from spacy.matcher import PhraseMatcher
from spacy.lang.en import English
from spacy.util import minibatch, compounding
from spacy.training import Example
#from spacy.gold import GoldParse 
from spacy.scorer import Scorer
import random

In [4]:
nlp = English()
#nlp = spacy.load("en_trf_bertbaseuncased_lg")
#nlp.vocab, attr='lemma', validate=True

In [5]:
#TITLE_matcher = PhraseMatcher(nlp.vocab, attr='LOWER')
TITLE_matcher = PhraseMatcher(nlp.vocab, attr='LOWER')
TITLE_patterns = [nlp(i) for i in final_df['Title'].tolist()]
TITLE_matcher.add("Title", None,*TITLE_patterns)

In [6]:
CLIENT_matcher = PhraseMatcher(nlp.vocab, attr='LOWER')
CLIENT_patterns = [nlp(i) for i in final_df['Client'].tolist()]
CLIENT_matcher.add("Client", None,*CLIENT_patterns)

In [7]:
SOCIAL_SECURITY_NO_matcher = PhraseMatcher(nlp.vocab, attr='LOWER')
SOCIAL_SECURITY_NO_patterns = [nlp(i) for i in final_df['SSN'].tolist()]
SOCIAL_SECURITY_NO_matcher.add("SSN", None,*SOCIAL_SECURITY_NO_patterns)

In [8]:
ADDRESS_matcher = PhraseMatcher(nlp.vocab, attr='LOWER')
ADDRESS_patterns = [nlp(i) for i in final_df['Address'].tolist()]
ADDRESS_matcher.add("Address", None,*ADDRESS_patterns)

In [9]:
TRAIN_TEXTS = final_df['Ext_text'].tolist()
#TEST_TEXTS = test['extracted_text'].tolist()

In [10]:
from tqdm import tqdm
TRAINING_DATA = []

# Create a Doc object for each text in TEXTS
for doc in tqdm(nlp.pipe(TRAIN_TEXTS)):
    # Match on the doc and create a list of matched spans
    TITLE_spans = [doc[start:end] for match_id, start, end in TITLE_matcher(doc)]
    CLIENT_spans = [doc[start:end] for match_id, start, end in CLIENT_matcher(doc)]
    ADDRESS_spans = [doc[start:end] for match_id, start, end in ADDRESS_matcher(doc)]
    SOCIAL_SECURITY_NO_spans = [doc[start:end] for match_id, start, end in SOCIAL_SECURITY_NO_matcher(doc)]
    
    # Get (start character, end character, label) tuples of matches
    entities = [(span.start_char, span.end_char, "TITLE") for span in TITLE_spans]
    entities.extend([(span.start_char, span.end_char, "CLIENT") for span in CLIENT_spans])
    entities.extend([(span.start_char, span.end_char, "ADDRESS") for span in ADDRESS_spans])
    entities.extend([(span.start_char, span.end_char, "SOCIAL_SECURITY_NO") for span in SOCIAL_SECURITY_NO_spans])

    # Format the matches as a (doc.text, entities) tuple
    training_example = (doc.text, {"entities": entities})
    # Append the example to the training data
    TRAINING_DATA.append(training_example)

5it [00:00, 331.06it/s]


In [11]:
# Remove repetated of same type entity
for i,_ in enumerate(TRAINING_DATA):
    TRAINING_DATA[i][1]['entities'] = ([next(b) for a, b in itertools.groupby( 
                         TRAINING_DATA[i][1]['entities'], lambda y: y[2])])

In [12]:
TRAINING_DATA[0]

('Reset Form TD Ameritrade ACCOUNT TRANSFER FORM TO TRANSFER YOUR EXISTING Institutional ACCOUNT TO TD AMERITRADE Account # 896542008 Advisor Code 806542008 Case # PLEASE ATTACH A COPY OF YOUR LATEST STATEMENT. 1 TD AMERITRADE ACCOUNT INFORMATION Account Title/Registration Alexis W Figueroa Social Security Number/Tax ID: 555455545 Clearing Number: 0188 Individual Personal Trust Limited Liability Company [] Roth IRA Qualified Retirement Plan (Non-Qualified) Employer Plan Type: Joint Estate Partnership/Investment Club SEP IRA Simple IRA UTMA/UGMA Corporate Traditional IRA/Rollover IRA Coverdell ESA Beneficiary IRA Other: 2 INFORMATION ON THE ACCOUNT YOU ARE TRANSFERRING FROM Account Title/Registration as shown on your statement: Account Number: Alexis W Figueroa 78965478 Name of Delivering Firm: Employee Fiduciary Physical Address of Firm (no PO BOX) 1376 Brentwood Drive, Austin, Texas, 78701 Telephone Number (required): (541) 754-3008 Individua Personal Trust Limited Liability Company R

In [13]:
model = None

n_iter=100
#load the model

if model is not None:
    nlp = spacy.load(model)  
    print("Loaded model '%s'" % model)
else:
    nlp = spacy.blank('en')  
    print("Created blank 'en' model")

#set up the pipeline

if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe('ner', last=True)
else:
    ner = nlp.get_pipe('ner')

for _, annotations in TRAINING_DATA:
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])

other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):  # only train NER
    optimizer = nlp.begin_training()
    for itn in range(n_iter):
        random.shuffle(TRAINING_DATA)
        losses = {}
        for text, annotations in tqdm(TRAINING_DATA):
            example = Example.from_dict(nlp.make_doc(text), annotations)
            nlp.update(
                [example], 
                drop=0.5,  
                sgd=optimizer,
                losses=losses)
        print(losses)

  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

Created blank 'en' model


 20%|████████████████▊                                                                   | 1/5 [00:00<00:00,  8.77it/s]

{'ner': 2176.3723727464676}


 20%|████████████████▊                                                                   | 1/5 [00:00<00:00,  5.97it/s]

...

{'ner': 0.00017779062747903555}


  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

{'ner': 0.033638700670338036}


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:01<00:00,  3.44it/s]

{'ner': 0.10176258565216728}


In [18]:
output_dir='finnermodel'
print(output_dir)
from pathlib import Path
# save model to output directory
if output_dir is not None:
    output_dir = Path(output_dir)
if not output_dir.exists():
    output_dir.mkdir()
nlp.to_disk(output_dir)
print("Saved model to", output_dir)
TEST_TEXTS = TRAIN_TEXTS

finnermodel
Saved model to finnermodel


In [19]:
# test the saved model
directory_of_saved_model = 'finnermodel'
output_dir = directory_of_saved_model
print("Loading from", output_dir)
nlp2 = spacy.load(output_dir)

def predict_entities(text, nlp2):
    # test the trained model
    doc = nlp2(text)
    entities = []
    for ent in doc.ents:
        entities.append((ent.text, ent.label_))
    return entities

Loading from finnermodel


In [20]:
out = predict_entities(TEST_TEXTS[2], nlp2)
out

[('ACCOUNT TRANSFER FORM', 'TITLE'),
 ('David T Jarvis', 'CLIENT'),
 ('372525158', 'SOCIAL_SECURITY_NO'),
 ('4572 Haven Lane, Lansing, Michigan,48933', 'ADDRESS')]

In [21]:
out = predict_entities(TEST_TEXTS[0], nlp2)
out

[('ACCOUNT TRANSFER FORM', 'TITLE'),
 ('Alexis W Figueroa', 'CLIENT'),
 ('555455545', 'SOCIAL_SECURITY_NO'),
 ('1376 Brentwood Drive, Austin, Texas, 78701', 'ADDRESS')]